In [1]:
import pandas as pd
import enchant   
from pandas import ExcelWriter
from pandas import ExcelFile
from pandas import DataFrame
from openpyxl import Workbook
import xlsxwriter
import math
from collections import Counter
import os

In [4]:
def read_excel(fileName, rowCount, worksheet, ML, successListWorksheet, nameCount, failuresListWorksheet, nameCountF):
    AL = pd.read_excel(fileName)
    columns_list = AL.columns

    AL.rename(columns = {columns_list[0]:'Attendee', 
                        'Unnamed: 1':'Score','Unnamed: 2':'    1:46 PM','Unnamed: 3':'        1:55 PM','Unnamed: 4':'     2:01 PM',
                        'Unnamed: 5':'       2:41 PM'},inplace=True)
    AL= AL.drop(AL.index [  0:4  ] )
    AL= AL.drop(AL.index [  0:2  ] )
    AL.drop(AL.tail(1).index,inplace=True)
    print(AL)
    master_count=0
    for i in ML.iterrows():
        master_count=master_count+1;
    print(master_count)

    Attendance_count=0
    directmatch=0
    cosineL=[]
    cosinematch=0
    count_levenshtein_match=0
    bas_name = os.path.basename(fileName)
    split_name = bas_name.split(".")[0]
    successListWorksheet.write(nameCount, 0, split_name)
    failuresListWorksheet.write(nameCountF, 0, split_name)
    nameCount = nameCount+1
    nameCountF = nameCountF+1
    countF = 0
    for i, row in AL.iterrows():
        Attendance_count+=1
        Attendeename=f"{row['Attendee']}"
        upper_case_AN=Attendeename.upper()
        sorted_Attendeename= ''.join(sorted(upper_case_AN))
        ans=900
        flag=0
        for j, row2 in ML.iterrows():
            Mastername=f"{row2['Name']}"
            if(Mastername==upper_case_AN):
                flag=1
                break
            
        if(flag==1):
            directmatch=directmatch+1
        else:
            cosineL.append(Attendeename)
            count_levenshtein_match+=1
    #print(cosineL, attencount, directmatch )

    levenshtein_threshold_0=0
    levenshtein_threshold_1=0
    levenshtein_remaining=0
    for i in cosineL:
        attendeename=i
        ans=500
        upper_case_attendeename=attendeename.upper()
        upper_case_removed=upper_case_attendeename.replace(" ", "")
        sorted_attendeename= ''.join(sorted(upper_case_removed))
        for j, row2 in ML.iterrows():
            Mastername=f"{row2['Name']}"
            Scholar_number = f"{row2['Redg_Number']}"
            master_name_removed=Mastername.replace(" ", "")
            sorted_mastername= ''.join(sorted(master_name_removed))
            cos= (enchant.utils.levenshtein(str(sorted_attendeename), str(sorted_mastername)))
            if(cos<=ans):
                ans=cos
                resultant_mastername= Mastername
                resultant_scholarnumber= Scholar_number
                resultant_master_name=sorted_mastername
        print(sorted_attendeename+"  "+resultant_master_name+"  "+attendeename+"   "+resultant_mastername+"  "+str(ans))
        
        if(ans==0):
            levenshtein_threshold_0+=1
            successListWorksheet.write(nameCount, 0, resultant_mastername)
            successListWorksheet.write(nameCount, 1, resultant_scholarnumber)
            nameCount = nameCount+1
        
        elif(ans==1):
            levenshtein_threshold_1+=1
            successListWorksheet.write(nameCount, 0, resultant_mastername)
            successListWorksheet.write(nameCount, 1, resultant_scholarnumber)
            nameCount = nameCount+1
        
        
        else:
            levenshtein_remaining+=1
            failuresListWorksheet.write(nameCountF, 0, attendeename)
            countF = countF+1
            nameCountF = nameCountF+1
            
    if countF == 0:
        failuresListWorksheet.write(nameCountF, 0, "No Failures")
        nameCountF = nameCountF+1
        
    #print(cosineS1,cosineS2,cosineF)
    nameCount = nameCount+1 
    nameCountF = nameCountF+1
            
     
    master_count
    x=Attendance_count/master_count
    y=x*100
    total_per=round(y,2)
    directmatch
    total_per
    per_dir_str_match=directmatch/Attendance_count 
    per_dir_str_match=per_dir_str_match*100
    per_dir_str_match=round(per_dir_str_match,2)
    lev_str_match=count_levenshtein_match/Attendance_count
    lev_str_match=lev_str_match*100
    lev_str_match=round(lev_str_match,2)
    lev_per_thr_0=levenshtein_threshold_0/Attendance_count
    lev_per_thr_0=lev_per_thr_0*100
    lev_per_thr_0=round(lev_per_thr_0,2)
    lev_per_thr_0
    lev_per_thr_1=levenshtein_threshold_1/Attendance_count
    lev_per_thr_1=lev_per_thr_1*100
    lev_per_thr_1=round(lev_per_thr_1,2)
    lev_per_thr_rem=levenshtein_remaining/Attendance_count
    lev_per_thr_rem=lev_per_thr_rem*100
    lev_per_thr_rem=round(lev_per_thr_rem,2)
            
    worksheet.write(0,0,'Date')
    worksheet.write(0, 1, 'Total percentage of attendee')
    worksheet.write(0, 2, 'Total percentage of direct string match' )
    worksheet.write(0, 3, 'Total percentage of levenshtein match' )
    worksheet.write(0, 4, 'percentage of levenshtein match with threshold as 0' )
    worksheet.write(0, 5, 'percentage of levenshtein match with threshold as 1' )
    worksheet.write(0, 6, 'Failure percentage(percentage of remaining levenshtein match)' )
    worksheet.write(0, 7, 'Number of Attendee')
    worksheet.write(0, 8, 'Number of direct string match')
    worksheet.write(0, 9, 'Number of levenshtein string match')
    worksheet.write(0, 10, 'Number of levenshtein match-0')
    worksheet.write(0, 11, 'Number of levenshtein match-1')
    worksheet.write(0, 12,'Number of levenshtein match-remaining')
    
    date_file = fileName.split(".")
    date_tbu = date_file[0]

    worksheet.write(rowCount,0,date_tbu)
    worksheet.write(rowCount, 1, total_per)
    worksheet.write(rowCount,2, per_dir_str_match )
    worksheet.write(rowCount, 3, lev_str_match)
    worksheet.write(rowCount, 4, lev_per_thr_0)
    worksheet.write(rowCount, 5, lev_per_thr_1)
    worksheet.write(rowCount, 6, lev_per_thr_rem)
    worksheet.write(rowCount, 7, Attendance_count )
    worksheet.write(rowCount, 8, directmatch)
    worksheet.write(rowCount, 9, levenshtein_threshold_0+levenshtein_threshold_1)
    worksheet.write(rowCount, 10, levenshtein_threshold_0)
    worksheet.write(rowCount, 11, levenshtein_threshold_1)
    worksheet.write(rowCount, 12, levenshtein_remaining)
    return (nameCount, nameCountF)

In [5]:
def checkIfXlsx(fileName):
    fileData = fileName.split(".")
    fileExt = fileData[-1]
    fileBaseName = os.path.basename(fileName)
    if(fileBaseName=='levenshtein_output_set_3.xlsx' or fileBaseName=='MasterStudentList-3.xlsx' or
        fileBaseName=='Output_Success.xlsx' or fileBaseName=='Output_Failures.xlsx'or fileBaseName=='$stats_final_layer-2_final.xlsx'):
        return False
    if(fileExt=='xlsx'):
        return True
    return False

if __name__ == '__main__':
    ML = pd.read_excel("MasterStudentList-3.xlsx")
    workbook = xlsxwriter.Workbook("levenshtein_output_set_3.xlsx",)
    successListWorkbook = xlsxwriter.Workbook("Output_Success.xlsx")
    failuresListWorkbook = xlsxwriter.Workbook("Output_Failures.xlsx")
    worksheet = workbook.add_worksheet()
    successListWorksheet = successListWorkbook.add_worksheet()
    failuresListWorksheet = failuresListWorkbook.add_worksheet()
    dir_list = os.listdir()
    fileCount = 0
    nameCount = 0
    nameCountF = 0
    for file in dir_list:
        if(checkIfXlsx(fileName=file)):
            print(file)
            fileCount = fileCount+1
            (camCount, camCount_f) = read_excel(file, fileCount, worksheet, ML, successListWorksheet, nameCount, failuresListWorksheet, nameCountF)
            nameCount = camCount
            nameCountF = camCount_f
    workbook.close()
    successListWorkbook.close()
    failuresListWorkbook.close()

2022-01-11.xlsx
                 Attendee Score     1:46 PM         1:55 PM
6              AMIT KUMAR   2/2           ✅               ✅
7   ANUMANCHIPALLI ADITYA   2/2           ✅               ✅
8            ANURAG KUMAR   2/2           ✅               ✅
9    CHENNAKESAVULA KUMAR   2/2           ✅               ✅
10           Gaurav KUMAR   2/2           ✅               ✅
11           GONDU GANESH   2/2           ✅               ✅
12         KARUSODU KUMAR   2/2           ✅               ✅
13       KOTTAPALLI KUMAR   2/2           ✅               ✅
14         MD IMRAN ANJUM   2/2           ✅               ✅
15           MITHUN KUMAR   2/2           ✅               ✅
16          NIRAKAR BHOLO   2/2           ✅               ✅
17        Niteshkumar Sah   2/2           ✅               ✅
18     PANKAJKUMAR SHARMA   2/2           ✅               ✅
19        POTRAKANTI SYAM   2/2           ✅               ✅
20         PRADHAN GANESH   2/2           ✅               ✅
21              RAJ ARYA

AAAJNRRY  AAAJNRRY  RAJ ARYAN   RAJ  ARYAN  0
AAAGJMRTU  AAAGJMRTU  RAJ GAUTAM   RAJ  GAUTAM  0
AADEEHIKMNPRRSTUY  AADEEHIKMNPRRSTUY  RITESH KUMAR PANDEY   RITESH KUMAR  PANDEY  0
AAAHHIJKNRSST  AAAHHIJKNRSST  Shashikant Raj   SHASHIKANT  RAJ  0
AAIKMMRTU  AAIKMMRTU  AMIT KUMAR   AMIT  KUMAR  0
AAEHHIKMNRSSTU  AAEHHIKMNRSSTU  Niteshkumar Sah   NITESH KUMAR  SAH  0
AAAIKMNOPRSTTY  AAAIKMNOPRSTTY  POTRAKANTI SYAM   POTRAKANTI  SYAM  0
AADEEKMNPRSU  AADEEKMNPRSU  SANDEEP KUMAR   SANDEEP  KUMAR  0
AAADDHHIMRSSTTY  AAADDHHIMRSSTTY  SATYAM SIDDHARTH   SATYAM  SIDDHARTH  0
2022-01-15.xlsx
                 Attendee Score     1:46 PM         1:55 PM      2:01 PM
6         Abhimanyu Kumar   3/3           ✅               ✅            ✅
7              AMIT KUMAR   3/3           ✅               ✅            ✅
8            ANURAG KUMAR   3/3           ✅               ✅            ✅
9          KARUSODU KUMAR   3/3           ✅               ✅            ✅
10           MITHUN KUMAR   3/3           ✅   

AAAAAHJKKMMNPRRSU  AAAAAHJKKMMNPRRSU  PANKAJKUMAR SHARMA   PANKAJ KUMAR  SHARMA  0
AADDEEIKLMPRSUU  AADDEEIKLMPRSUU  PEDIREDLA KUSUMA   PEDIREDLA  KUSUMA  0
AAAIKMNOPRSTTY  AAAIKMNOPRSTTY  POTRAKANTI SYAM   POTRAKANTI  SYAM  0
AAAJNRRY  AAAJNRRY  RAJ ARYAN   RAJ  ARYAN  0
AAADDHHIMRSSTTY  AAADDHHIMRSSTTY  SATYAM SIDDHARTH   SATYAM  SIDDHARTH  0
AAAHHIJKNRSST  AAAHHIJKNRSST  Shashikant Raj   SHASHIKANT  RAJ  0
AAAEHIKMNPRSTV  AAAAAEHIJKMNPRSTVY  VEMPATI SHANKAR   VEMPATI JAYA SHANKAR  4
2022-01-19.xlsx
                 Attendee Score     1:46 PM         1:55 PM      2:01 PM  \
6         Abhimanyu Kumar   4/4           ✅               ✅            ✅   
7              AMIT KUMAR   4/4           ✅               ✅            ✅   
8             ANKIT KUMAR   4/4           ✅               ✅            ✅   
9             BABLU KUMAR   4/4           ✅               ✅            ✅   
10   CHENNAKESAVULA KUMAR   4/4           ✅               ✅            ✅   
11            CHIRANJEE V   3/4      

AAIKMMRTU  AAIKMMRTU  AMIT KUMAR   AMIT  KUMAR  0
ADEGGHNNOSU  ADEGGHNNOSU  GONDU GANESH   GONDU  GANESH  0
AADKKMORRSUUU  AAAGKMRRUUV  KARUSODU KUMAR   GAURAV  KUMAR  5
ADHIIIKMNRU  ADHIIIKMNRU  NIDHI KUMARI   NIDHI  KUMARI  0
AABHIKLNOORR  AABHIKLNOORR  NIRAKAR BHOLO   NIRAKAR  BHOLO  0
AAEHHIKMNRSSTU  AAEHHIKMNRSSTU  Niteshkumar Sah   NITESH KUMAR  SAH  0
AADDEEIKLMPRSUU  AADDEEIKLMPRSUU  PEDIREDLA KUSUMA   PEDIREDLA  KUSUMA  0
AAHKLMRRUU  AAHKLMRRUU  RAHUL KUMAR   RAHUL  KUMAR  0
AAAJNRRY  AAAJNRRY  RAJ ARYAN   RAJ  ARYAN  0
AAAAHHKKMNORRSSSTUU  AAAAHHKKMNORRSSSTUU  SANTOSH KUMAR SAHUKAR   SANTOSH  KUMAR SAHUKAR  0
AAGHHIIKMNRSSSTU  AAGHHIIKMNRSSSTU  SATISHKUMAR SINGH   SATISH KUMAR  SINGH  0
AAADDHHIMRSSTTY  AAADDHHIMRSSTTY  SATYAM SIDDHARTH   SATYAM  SIDDHARTH  0
AAAHHIJKNRSST  AAAHHIJKNRSST  Shashikant Raj   SHASHIKANT  RAJ  0
AAHIIJKNRSUVYY  AAHIIIJKNRRSSUVYY  VYSYARAJU NIKHI   VYSYARAJU SRI NIKHI  3
AAAAACDHIIILLMNNPTUY  AAAAAAACDEHIIIKLLMNNNPTTUVY  ANUMANCHIPALLI ADITYA   ANU

AAAAACDHIIILLMNNPTUY  AAAAAAACDEHIIIKLLMNNNPTTUVY  ANUMANCHIPALLI ADITYA   ANUMANCHIPALLI VENKATA ADITYA  7
AABBKLMRUU  AABBKLMRUU  BABLU KUMAR   BABLU  KUMAR  0
ABDIIIKNNOR  ABDIIIKNNOR  BINODINI KAR   BINODINI  KAR  0
AAAACEEHKKLMNNRSUUV  AAAADKKLMNORRSUUUV  CHENNAKESAVULA KUMAR   KARUSODU  LAVANKUMAR  7
ADEGGHNNOSU  ADEGGHNNOSU  GONDU GANESH   GONDU  GANESH  0
AIIJKMORTUY  AIIJKMORTUY  JYOTI KUMARI   JYOTI  KUMARI  0
AADKKMORRSUUU  AAAGKMRRUUV  KARUSODU KUMAR   GAURAV  KUMAR  5
AAAHIIKKNORRT  AAAHIIKKNORRT  KOTRA NIHARIKA   KOTRA  NIHARIKA  0
AAHIKMMNRSU  AAHIKMMNRSU  MANISH KUMAR   MANISH  KUMAR  0
AAAAABDLNNRTV  AAAAABDLNNRTV  NAV RATNA BADAL   NAVRATNA  BADAL  0
ADHIIIKMNRU  ADHIIIKMNRU  NIDHI KUMARI   NIDHI  KUMARI  0
AAEHHIKMNRSSTU  AAEHHIKMNRSSTU  Niteshkumar Sah   NITESH KUMAR  SAH  0
AAAAAHJKKMMNPRRSU  AAAAAHJKKMMNPRRSU  PANKAJKUMAR SHARMA   PANKAJ KUMAR  SHARMA  0
AADDEEIKLMPRSUU  AADDEEIKLMPRSUU  PEDIREDLA KUSUMA   PEDIREDLA  KUSUMA  0
AAAIKMNOPRSTTY  AAAIKMNOPRSTTY  POTRA

AAAGKMNRRUU  AAAGKMNRRUU  ANURAG KUMAR   ANURAG  KUMAR  0
AAAACEEHKKLMNNRSUUV  AAAADKKLMNORRSUUUV  CHENNAKESAVULA KUMAR   KARUSODU  LAVANKUMAR  7
ADEGGHNNOSU  ADEGGHNNOSU  GONDU GANESH   GONDU  GANESH  0
AADKKMORRSUUU  AAAGKMRRUUV  KARUSODU KUMAR   GAURAV  KUMAR  5
AAEHHIKMNRSSTU  AAEHHIKMNRSSTU  Niteshkumar Sah   NITESH KUMAR  SAH  0
AAILPRSY  AAILPRSY  P SRILAYA   P  SRILAYA  0
AADDEEIKLMPRSUU  AADDEEIKLMPRSUU  PEDIREDLA KUSUMA   PEDIREDLA  KUSUMA  0
AAADEGHHNNPRS  AAADEGHHNNPRS  PRADHAN GANESH   PRADHAN  GANESH  0
AADEEHIKMNPRRSTUY  AADEEHIKMNPRRSTUY  RITESH KUMAR PANDEY   RITESH KUMAR  PANDEY  0
AAADDHHIMRSSTTY  AAADDHHIMRSSTTY  SATYAM SIDDHARTH   SATYAM  SIDDHARTH  0
AAAEHIKMNPRSTV  AAAAAEHIJKMNPRSTVY  VEMPATI SHANKAR   VEMPATI JAYA SHANKAR  4
AAHIIJKNRSUVYY  AAHIIIJKNRRSSUVYY  VYSYARAJU NIKHI   VYSYARAJU SRI NIKHI  3
AABHIKLNOORR  AABHIKLNOORR  NIRAKAR BHOLO   NIRAKAR  BHOLO  0
AAAAAHJKKMMNPRRSU  AAAAAHJKKMMNPRRSU  PANKAJKUMAR SHARMA   PANKAJ KUMAR  SHARMA  0
ADHIINNOOV  ADHIIINN

AAAGKMNRRUU  AAAGKMNRRUU  ANURAG KUMAR   ANURAG  KUMAR  0
AABBKLMRUU  AABBKLMRUU  BABLU KUMAR   BABLU  KUMAR  0
AAAACEEHKKLMNNRSUUV  AAAADKKLMNORRSUUUV  CHENNAKESAVULA KUMAR   KARUSODU  LAVANKUMAR  7
AAAGGLNNRTUU  AAAGGLNNRTUU  GANGULA TARUN   GANGULA  TARUN  0
ADEGGHNNOSU  ADEGGHNNOSU  GONDU GANESH   GONDU  GANESH  0
AIIJKMORTUY  AIIJKMORTUY  JYOTI KUMARI   JYOTI  KUMARI  0
AADKKMORRSUUU  AAAGKMRRUUV  KARUSODU KUMAR   GAURAV  KUMAR  5
AAHIKKMNRSU  AAHIKKMNRSU  KISHAN KUMAR   KISHAN  KUMAR  0
AAAHIIKKNORRT  AAAHIIKKNORRT  KOTRA NIHARIKA   KOTRA  NIHARIKA  0
AADIJMMMNNRU  AADIJMMMNNRU  MD IMRAN ANJUM   MDIMRAN  ANJUM  0
AABHIKLNOORR  AABHIKLNOORR  NIRAKAR BHOLO   NIRAKAR  BHOLO  0
AAEHHIKMNRSSTU  AAEHHIKMNRSSTU  Niteshkumar Sah   NITESH KUMAR  SAH  0
AAAAAHJKKMMNPRRSU  AAAAAHJKKMMNPRRSU  PANKAJKUMAR SHARMA   PANKAJ KUMAR  SHARMA  0
AADDEEIKLMPRSUU  AADDEEIKLMPRSUU  PEDIREDLA KUSUMA   PEDIREDLA  KUSUMA  0
AAADEGHHNNPRS  AAADEGHHNNPRS  PRADHAN GANESH   PRADHAN  GANESH  0
AAHKLMRRUU  AAHKL

AAADEGHHNNPRS  AAADEGHHNNPRS  PRADHAN GANESH   PRADHAN  GANESH  0
AAAJNRRY  AAAJNRRY  RAJ ARYAN   RAJ  ARYAN  0
AAGHHIIKMNRSSSTU  AAGHHIIKMNRSSSTU  SATISHKUMAR SINGH   SATISH KUMAR  SINGH  0
AAAHHIJKNRSST  AAAHHIJKNRSST  Shashikant Raj   SHASHIKANT  RAJ  0
AAKMMRTTUU  AAKMMRTTUU  UTTAM KUMAR   UTTAM  KUMAR  0
2022-03-11.xlsx
                   Attendee Score     1:46 PM         1:55 PM
6           Abhimanyu Kumar   2/2           ✅               ✅
7   Aishwarya Ankush Sawant   2/2           ✅               ✅
8                AMIT KUMAR   2/2           ✅               ✅
9               ANKIT KUMAR   2/2           ✅               ✅
10    ANUMANCHIPALLI ADITYA   2/2           ✅               ✅
11             ANURAG KUMAR   2/2           ✅               ✅
12             BINODINI KAR   2/2           ✅               ✅
13     CHENNAKESAVULA KUMAR   1/2           ✅               ❎
14             GONDU GANESH   2/2           ✅               ✅
15           KARUSODU KUMAR   2/2           ✅        